
<br>
=======================<br>
Morphological Filtering<br>
=======================<br>
Morphological image processing is a collection of non-linear operations related<br>
to the shape or morphology of features in an image, such as boundaries,<br>
skeletons, etc. In any given technique, we probe an image with a small shape or<br>
template called a structuring element, which defines the region of interest or<br>
neighborhood around a pixel.<br>
In this document we outline the following basic morphological operations:<br>
1. Erosion<br>
2. Dilation<br>
3. Opening<br>
4. Closing<br>
5. White Tophat<br>
6. Black Tophat<br>
7. Skeletonize<br>
8. Convex Hull<br>
To get started, let's load an image using ``io.imread``. Note that morphology<br>
functions only work on gray-scale or binary images, so we set ``as_gray=True``.<br>


In [ ]:
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import img_as_ubyte

In [ ]:
orig_phantom = img_as_ubyte(data.shepp_logan_phantom())
fig, ax = plt.subplots()
ax.imshow(orig_phantom, cmap=plt.cm.gray)

####################################################################<br>
Let's also define a convenience function for plotting comparisons:

In [ ]:
def plot_comparison(original, filtered, filter_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')

####################################################################<br>
Erosion<br>
=======<br>
<br>
Morphological ``erosion`` sets a pixel at (i, j) to the *minimum over all<br>
pixels in the neighborhood centered at (i, j)*. The structuring element,<br>
``selem``, passed to ``erosion`` is a boolean array that describes this<br>
neighborhood. Below, we use ``disk`` to create a circular structuring<br>
element, which we use for most of the following examples.

In [ ]:
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)
from skimage.morphology import black_tophat, skeletonize, convex_hull_image  # noqa
from skimage.morphology import disk  # noqa

In [ ]:
selem = disk(6)
eroded = erosion(orig_phantom, selem)
plot_comparison(orig_phantom, eroded, 'erosion')

####################################################################<br>
Notice how the white boundary of the image disappears or gets eroded as we<br>
increase the size of the disk. Also notice the increase in size of the two<br>
black ellipses in the center and the disappearance of the 3 light gray<br>
patches in the lower part of the image.<br>
<br>
Dilation<br>
========<br>
<br>
Morphological ``dilation`` sets a pixel at (i, j) to the *maximum over all<br>
pixels in the neighborhood centered at (i, j)*. Dilation enlarges bright<br>
regions and shrinks dark regions.

In [ ]:
dilated = dilation(orig_phantom, selem)
plot_comparison(orig_phantom, dilated, 'dilation')

####################################################################<br>
Notice how the white boundary of the image thickens, or gets dilated, as we<br>
increase the size of the disk. Also notice the decrease in size of the two<br>
black ellipses in the centre, and the thickening of the light gray circle<br>
in the center and the 3 patches in the lower part of the image.<br>
<br>
Opening<br>
=======<br>
<br>
Morphological ``opening`` on an image is defined as an *erosion followed by<br>
a dilation*. Opening can remove small bright spots (i.e. "salt") and<br>
connect small dark cracks.

In [ ]:
opened = opening(orig_phantom, selem)
plot_comparison(orig_phantom, opened, 'opening')

####################################################################<br>
Since ``opening`` an image starts with an erosion operation, light regions<br>
that are *smaller* than the structuring element are removed. The dilation<br>
operation that follows ensures that light regions that are *larger* than<br>
the structuring element retain their original size. Notice how the light<br>
and dark shapes in the center their original thickness but the 3 lighter<br>
patches in the bottom get completely eroded. The size dependence is<br>
highlighted by the outer white ring: The parts of the ring thinner than the<br>
structuring element were completely erased, while the thicker region at the<br>
top retains its original thickness.<br>
<br>
Closing<br>
=======<br>
<br>
Morphological ``closing`` on an image is defined as a *dilation followed by<br>
an erosion*. Closing can remove small dark spots (i.e. "pepper") and<br>
connect small bright cracks.<br>
<br>
To illustrate this more clearly, let's add a small crack to the white<br>
border:

In [ ]:
phantom = orig_phantom.copy()
phantom[10:30, 200:210] = 0

In [ ]:
closed = closing(phantom, selem)
plot_comparison(phantom, closed, 'closing')

####################################################################<br>
Since ``closing`` an image starts with an dilation operation, dark regions<br>
that are *smaller* than the structuring element are removed. The dilation<br>
operation that follows ensures that dark regions that are *larger* than the<br>
structuring element retain their original size. Notice how the white<br>
ellipses at the bottom get connected because of dilation, but other dark<br>
region retain their original sizes. Also notice how the crack we added is<br>
mostly removed.<br>
<br>
White tophat<br>
============<br>
<br>
The ``white_tophat`` of an image is defined as the *image minus its<br>
morphological opening*. This operation returns the bright spots of the<br>
image that are smaller than the structuring element.<br>
<br>
To make things interesting, we'll add bright and dark spots to the image:

In [ ]:
phantom = orig_phantom.copy()
phantom[340:350, 200:210] = 255
phantom[100:110, 200:210] = 0

In [ ]:
w_tophat = white_tophat(phantom, selem)
plot_comparison(phantom, w_tophat, 'white tophat')

####################################################################<br>
As you can see, the 10-pixel wide white square is highlighted since it is<br>
smaller than the structuring element. Also, the thin, white edges around<br>
most of the ellipse are retained because they're smaller than the<br>
structuring element, but the thicker region at the top disappears.<br>
<br>
Black tophat<br>
============<br>
<br>
The ``black_tophat`` of an image is defined as its morphological **closing<br>
minus the original image**. This operation returns the *dark spots of the<br>
image that are smaller than the structuring element*.

In [ ]:
b_tophat = black_tophat(phantom, selem)
plot_comparison(phantom, b_tophat, 'black tophat')

####################################################################<br>
As you can see, the 10-pixel wide black square is highlighted since<br>
it is smaller than the structuring element.<br>
<br>
**Duality**<br>
<br>
As you should have noticed, many of these operations are simply the reverse<br>
of another operation. This duality can be summarized as follows:<br>
<br>
  1. Erosion <-> Dilation<br>
<br>
  2. Opening <-> Closing<br>
<br>
  3. White tophat <-> Black tophat<br>
<br>
Skeletonize<br>
===========<br>
<br>
Thinning is used to reduce each connected component in a binary image to a<br>
*single-pixel wide skeleton*. It is important to note that this is<br>
performed on binary images only.

In [ ]:
horse = data.horse()

In [ ]:
sk = skeletonize(horse == 0)
plot_comparison(horse, sk, 'skeletonize')

####################################################################<br>
As the name suggests, this technique is used to thin the image to 1-pixel<br>
wide skeleton by applying thinning successively.<br>
<br>
Convex hull<br>
===========<br>
<br>
The ``convex_hull_image`` is the *set of pixels included in the smallest<br>
convex polygon that surround all white pixels in the input image*. Again<br>
note that this is also performed on binary images.

In [ ]:
hull1 = convex_hull_image(horse == 0)
plot_comparison(horse, hull1, 'convex hull')

####################################################################<br>
As the figure illustrates, ``convex_hull_image`` gives the smallest polygon<br>
which covers the white or True completely in the image.<br>
<br>
If we add a small grain to the image, we can see how the convex hull adapts<br>
to enclose that grain:

In [ ]:
horse_mask = horse == 0
horse_mask[45:50, 75:80] = 1

In [ ]:
hull2 = convex_hull_image(horse_mask)
plot_comparison(horse_mask, hull2, 'convex hull')

####################################################################<br>
Additional Resources<br>
====================<br>
<br>
1. `MathWorks tutorial on morphological processing<br>
<https://se.mathworks.com/help/images/morphological-dilation-and-erosion.html>`_<br>
<br>
2. `Auckland university's tutorial on Morphological Image Processing<br>
<https://www.cs.auckland.ac.nz/courses/compsci773s1c/lectures/ImageProcessing-html/topic4.htm>`_<br>
<br>
3. https://en.wikipedia.org/wiki/Mathematical_morphology

In [ ]:
plt.show()